# Torchvision

Torchvision 是一个和 PyTorch 配合使用的 Python 包，包含很多图像处理的工具。

流程：
数据的读取、网络的设计、优化方法与损失函数的选择以及一些辅助的工具等。


In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision

## PyTorch 的数据读取

### DataSet 类

无论使用自定义的数据集，还是官方封装好的数据集，其本质都是继承了 Dataset 类。而在继承 Dataset 类时，至少需要重写以下几个方法：
- `__init__()`：构造函数，可自定义数据读取方法以及进行数据预处理；
- `__len__()`：返回数据集大小；
- `__getitem__()`：索引数据集中的某一个数据。


In [2]:
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)

    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]


在构造函数中，传入 Tensor 类型的数据与标签；在 `__len__` 函数中，直接返回 Tensor 的大小；在 `__getitem__` 函数中返回索引的数据与标签。


In [3]:
# 生成数据
data_tensor = torch.randn(10, 3)
target_tensor = torch.randint(2, (10,))

print(data_tensor)
print(target_tensor)

# 将数据封装成Dataset
my_dataset = MyDataset(data_tensor, target_tensor)

print('Dataset size:', len(my_dataset))
print('tensor_data[0]: ', my_dataset[0])

tensor([[-1.3045,  1.4716, -0.0198],
        [ 0.1322,  0.8875,  0.0471],
        [-0.7870,  0.4271, -0.3337],
        [ 0.1074, -1.1179, -0.8699],
        [-1.8975, -0.8072,  1.3093],
        [-0.5782, -0.0682, -0.2453],
        [-0.1306, -1.0480,  1.4519],
        [-0.8955,  1.7510,  1.6876],
        [-1.4361,  0.1746, -0.4942],
        [-0.5906, -1.6072, -2.2137]])
tensor([1, 0, 1, 0, 0, 1, 0, 0, 0, 1])
Dataset size: 10
tensor_data[0]:  (tensor([-1.3045,  1.4716, -0.0198]), tensor(1))


### DataLoader 类

在实际项目中，如果数据量很大，考虑到内存有限、I/O 速度等问题，在训练过程中不可能一次性的将所有数据全部加载到内存中，也不能只用一个进程去加载，所以就需要多进程、迭代加载，而 DataLoader 就是基于这些需要被设计出来的。DataLoader 是一个迭代器，最基本的使用方法就是传入一个 Dataset 对象，它会根据参数 batch_size 的值生成一个 batch 的数据，节省内存的同时，它还可以实现多进程、数据打乱等处理。

In [4]:
tensor_dataloader = DataLoader(dataset=my_dataset, batch_size=2, shuffle=True, num_workers=0)

for data, target in tensor_dataloader:
    print(data, target)

print('One batch tensor data: ', iter(tensor_dataloader).next())

tensor([[-1.4361,  0.1746, -0.4942],
        [ 0.1074, -1.1179, -0.8699]]) tensor([0, 0])
tensor([[ 0.1322,  0.8875,  0.0471],
        [-0.5906, -1.6072, -2.2137]]) tensor([0, 1])
tensor([[-1.3045,  1.4716, -0.0198],
        [-0.1306, -1.0480,  1.4519]]) tensor([1, 0])
tensor([[-0.7870,  0.4271, -0.3337],
        [-1.8975, -0.8072,  1.3093]]) tensor([1, 0])
tensor([[-0.5782, -0.0682, -0.2453],
        [-0.8955,  1.7510,  1.6876]]) tensor([1, 0])
One batch tensor data:  [tensor([[-1.4361,  0.1746, -0.4942],
        [ 0.1074, -1.1179, -0.8699]]), tensor([0, 0])]


- dataset：Dataset 类型，输入的数据集，必须参数；
- batch_size：int 类型，每个 batch 有多少个样本；
- shuffle：bool 类型，在每个 epoch 开始的时候，是否对数据进行重新打乱；
- num_workers：int 类型，加载数据的进程数，0 意味着所有的数据都会被加载进主进程，默认为 0。

## Torchvision

PyTroch 官方提供了一些常用的图片数据集 —— Torchvision 。
它是一个和 PyTorch 配合使用的 Python 包，集合了常用数据集 + 常见网络模型 + 常用图像处理方法。

[torchvision.datasets包所有支持的数据集](https://pytorch.org/vision/stable/datasets.html)

### MNIST 数据集

MNIST 数据集是一个著名的手写数字数据集，上手简单，在深度学习领域，手写数字识别是一个很经典的学习入门样例。

[数据集下载](http://yann.lecun.com/exdb/mnist/)

| 数据集 | 描述 | 字节数 |
|---|---|---|
| train-images-idx3-ubyte.gz | training set images | 9912422 bytes |
| train-labels-idx1-ubyte.gz | training set labels | 28881 bytes |
| t10k-images-idx3-ubyte.gz | test set images | 1648877 bytes |
| t10k-labels-idx1-ubyte.gz | test set labels | 4542 bytes |

数据加载

In [5]:
# 先去手动解压 ds/MNIST/raw 下的文件，或者直接改 download=True
mnist_dataset = torchvision.datasets.MNIST(root='./ds',
                                           train=True,
                                           transform=None,
                                           target_transform=None,
                                           download=False)

mnist_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./ds
    Split: Train

torchvision.datasets.MNIST是一个类，对它进行实例化，即可返回一个 MNIST 数据集对象。构造函数包括包含 5 个参数：
- root：是一个字符串，用于指定你想要保存 MNIST 数据集的位置。如果 download 是 Flase，则会从目标位置读取数据集；
- download：是布尔类型，表示是否下载数据集。如果为 True，则会自动从网上下载这个数据集，存储到 root 指定的位置。如果指定位置已经存在数据集文件，则不会重复下载；
- train：是布尔类型，表示是否加载训练集数据。如果为 True，则只加载训练数据。如果为 False，则只加载测试数据集。这里需要注意，并不是所有的数据集都做了训练集和测试集的划分，这个参数并不一定是有效参数，具体需要参考官方接口说明文档；
- transform：用于对图像进行预处理操作，例如数据增强、归一化、旋转或缩放等。这些操作我们会在下节课展开讲解；
- target_transform：用于对图像标签进行预处理操作。

没有官方接口的图像数据集，也可以使用以 torchvision.datasets.ImageFolder 接口来自行定义。


数据预览

In [9]:
mnist_dataset_list = list(mnist_dataset)
item = mnist_dataset_list[0]
print(item)
print(type(item))
print(len(item))

(<PIL.Image.Image image mode=L size=28x28 at 0x7F9BD164CE50>, 5)
<class 'tuple'>
2


看到上面的图像数据是 `PIL.Image.Image` 类型的，这种类型可以直接在 Jupyter 中显示出来。

In [10]:
from IPython.display import display

display(item[0])
print("Image label is:", item[1])


Image label is: 5
